In [2]:
import os
from pathlib import Path
import re
import pandas as pd
import geopandas as gpd
import folium
from urllib.request import urlretrieve
from zipfile import ZipFile

Download files from https://download.geofabrik.de/:

In [11]:
ab_url = "https://download.geofabrik.de/north-america/canada/alberta-latest-free.shp.zip"
urlretrieve(ab_url, "../data/ab-osm.zip")

('../data/ab-osm.zip', <http.client.HTTPMessage at 0x731289b3bed0>)

In [12]:
with ZipFile("../data/ab-osm.zip", "r") as zObject:
    zObject.extractall(path="../data/raw/osm-data")

In [18]:
osm = list()
p = Path("../data/raw/osm-data/")
for i in p.glob("*.shp"):
    df = gpd.read_file(i, engine="pyogrio")
    key_name = re.sub("gis_osm_|_a|_free_1|.shp", "", i.name)
    df["key"] = key_name
    osm.append(df)

osm = pd.concat(osm)
osm = gpd.GeoDataFrame(osm)
osm = osm.loc[:, ["key", "fclass", "name", "geometry"]]
osm = osm.rename({"fclass": "value"}, axis=1)
osm.head()

,key,value,name,geometry
0,landuse,park,Nose Hill Park,"POLYGON ((-114.13938 51.10938, -114.13937 51.1..."
1,landuse,park,West Confederation Park,"POLYGON ((-114.11515 51.08155, -114.11496 51.0..."
2,landuse,park,Cochrane Ranche,"POLYGON ((-114.48593 51.19858, -114.48541 51.1..."
3,landuse,park,Mitford Park,"POLYGON ((-114.49651 51.19028, -114.49649 51.1..."
4,landuse,park,W.D. Kerfoot Park,"POLYGON ((-114.47413 51.19595, -114.47371 51.1..."


In [27]:
try:
    os.makedirs("../data/processed")
except FileExistsError:
    pass

osm.to_file("../data/processed/osm.gpkg", driver="GPKG")

In [6]:
osm = gpd.read_file("../data/processed/osm.gpkg", engine="pyogrio")
schema = pd.read_csv("../data/tables/osm-lulc.csv")
schema = schema.dropna(subset="lulc")
schema

,key,value,lulc
0,building,allotment_house,urban-low-density
1,building,annexe,urban-low-density
2,building,apartments,urban-low-density
3,building,bakehouse,commerical/residential/industrial
4,building,barn,developed-open-space
...,...,...,...
155,landuse,allotments,cropland
156,landuse,orchard,cropland
157,landuse,vineyard,cropland
158,landuse,plant_nursery,cropland


In [9]:
schema = pd.read_csv("../data/tables/osm-lulc.csv")
schema = schema.dropna(subset="lulc")
schema

,key,value,lulc
0,building,allotment_house,urban-low-density
1,building,annexe,urban-low-density
2,building,apartments,urban-low-density
3,building,bakehouse,commerical/residential/industrial
4,building,barn,developed-open-space
...,...,...,...
155,landuse,allotments,cropland
156,landuse,orchard,cropland
157,landuse,vineyard,cropland
158,landuse,plant_nursery,cropland


In [7]:
osm_lulc = osm.merge(schema, on=["key", "value"], how="inner")

In [14]:
osm_lulc.to_file("../data/processed/osm-lulc.gpkg",
                 driver="GPKG",
                 engine="pyogrio")

In [13]:
geom = osm_lulc.loc[osm_lulc.lulc == "urban-low-density"]
geom = geom.to_json()
geo_j = folium.GeoJson(geom)
m = folium.Map(tiles="CartoDB Positron")
geo_j.add_to(m)
m